# 활동량 데이터 전처리

In [2]:
import pandas as pd

In [ ]:
# 데이터 불러오기
df = pd.read_csv('data/raw/KS_AREA_ACCTO_PUBLIC_ALSFC_SUPPLY_STTUS_INFO_202407.csv')
df.head()

,BASE_YM,CTPRVN_CD,CTPRVN_NM,SIGNGU_CD,SIGNGU_NM,SIGNGU_ACCTO_FCLTY_CO,SIGNGU_ACCTO_POPLTN_CO,PSNBY_FCLTY_CO,PSNBY_FCL_CO_RANK_CO
0,202008,1100000000,서울특별시,1111000000,종로구,81,149962,0.00054,127
1,202008,1100000000,서울특별시,1114000000,중구,26,125990,0.00021,182
2,202008,1100000000,서울특별시,1117000000,용산구,28,229362,0.00012,207
3,202008,1100000000,서울특별시,1120000000,성동구,28,296183,0.00009,213
4,202008,1100000000,서울특별시,1121500000,광진구,25,348647,0.00007,220


지역 필터링

In [5]:
#서울,경기,인천 필터링
activity_df = df[(df['CTPRVN_NM'].isin(['서울특별시', '경기도', '인천광역시']))]
activity_df['CTPRVN_NM'].unique()

array(['서울특별시', '인천광역시', '경기도'], dtype=object)

In [ ]:
# 필요없는 컬럼 삭제
activity_df.drop(['CTPRVN_CD','SIGNGU_CD','PSNBY_FCL_CO_RANK_CO'], axis=1, inplace=True)

년도 필터링

In [7]:
activity_df['BASE_YM'].dtypes

dtype('int64')

In [8]:
# 2020년 필터링
activity_df= activity_df[activity_df['BASE_YM'].astype(str).str.startswith('2020')]

In [9]:
# 컬럼명 수정
activity_df.rename({'BASE_YM':'기준연월','CTPRVN_NM':'시도명','SIGNGU_NM':'시군구명','SIGNGU_ACCTO_FCLTY_CO':'시군구별시설수','SIGNGU_ACCTO_POPLTN_CO':'시군구별인구수','PSNBY_FCLTY_CO':'인당시설수'},
                     axis='columns', inplace=True)

시군구 통합

시군구2 : 경기/인천 '구' 단위 → '시' 단위 통합

In [10]:
def replace_values_with_multiple_substrings(activity_df, column, replacements):
  for old_value, new_value in replacements.items():
        activity_df.loc[activity_df[column].str.contains(old_value, na=False), column] = new_value
  return activity_df

replacements = {
    '수원시': '수원시',
    '성남시': '성남시',
    '안양시': '안양시',
    '부천시': '부천시',
    '안산시': '안산시',
    '고양시': '고양시',
    '용인시': '용인시',
}
activity_df['시군구2'] = activity_df['시군구명']
activity_df = replace_values_with_multiple_substrings(activity_df, '시군구2', replacements)

데이터 저장

In [13]:
activity_df.to_csv('data1/activity.csv')